In [ ]:
import multiprocessing as mp
import time
import random
import sys
import os
import errno
import shutil 
import subprocess
import numpy as np
import math
import matplotlib as mpl
import cma
from decimal import Decimal
import numpy as np
from astropy import units as u
from poliastro.bodies import Earth, Sun
from poliastro.twobody import Orbit
from poliastro.examples import molniya
from poliastro.plotting import plot
from poliastro.plotting import OrbitPlotter
import matplotlib.pyplot as plt
from numpy import linalg as LA
%matplotlib inline


cmu_mu = 10
pop_size = 50
no_of_impulses = 1
# Calculate the objective function based on these state vectors
choice = 1 # 1 for keplerian testing and 2 for state vectors


#define the state vector at time t= 0 in keplerian elements
initial_state_vector = np.array([7000.00,0,90,0,0,0])
#initial_state_vector = np.array([7050.00,0.03,98.00,0,0,0])


def kep2cart(X,nu):
    a = X[0] * u.km
    ecc = X[1] * u.one
    inc = np.mod(X[2],180) * u.deg
    raan = X[3] * u.deg
    argp = X[4] * u.deg
    nu = nu * u.deg
    #print('kep2cart value ofX : {}\n'.format(X))
    ss = Orbit.from_classical(Earth, a, ecc, inc, raan, argp, nu)
    #print('Radial : {0}, Velocity: {1}.\n'.format(ss.state.r,ss.state.v))
    r = ss.state.r
    v = ss.state.v
    sv_cart = np.append(r.value,v.value)
    return(sv_cart)

def cart2kep(r,v):
    r = r*u.km
    v = v*u.km/u.second
   # print('Position : {0} and Velocity : {1}.\n'.format(r,v))
    tmp = Orbit.from_vectors(Earth,r,v)
    sv = np.zeros(6)
    sv = [tmp.a.value,tmp.ecc.value,tmp.inc.value*(180/np.pi),tmp.raan.value*(180/np.pi), \
          tmp.argp.value*(180/np.pi),tmp.nu.value*(180/np.pi)]
    #print('Hello sv : {}\n'.format(sv))
    return(sv)
    

def make_sure_path_exists(path):
    try:
        os.makedirs(path)
    except OSError as exception:
        if exception.errno != errno.EEXIST:
            raise
            
def folder_already_exists_check(path):
    if os.path.exists(path):
        remove_temp_dir(path)
            
            
def remove_temp_dir(path):
    shutil.rmtree(path, ignore_errors=True)


def fitness_calculator(process_number, queue, x, X ):
 
    #for test (hohmann transfer)
    start = time.time()
    #print("Process {} has started at {}".format(process_number, start))
    sys.stdout.flush()
    
    #print('final Keplerian vector: {0} at {1}\n'.format(x,process_number))

    #if process_number == 1:
        #print('final vector: {0} at {1}\n'.format(x,process_number))
    #print('Norm_r: {0} Norm_v: {1}\n'.format(LA.norm(x[0:3]),LA.norm(x[3:6])))

    if choice == 1:
        sv_kep = cart2kep(x[0:3], x[3:6])
        #print(X)
        deltav = (np.absolute(LA.norm(X[0:3])) + np.absolute(LA.norm(X[4:7])))
   # For case 1 
        #a_error = np.absolute(sv_kep[0]-42164)
        #e_error = np.absolute((sv_kep[1]) - 0)
        #i_error =  np.absolute((sv_kep[2] - 90))
        #raan_error = np.absolute((sv_kep[3]) - 0)
        #print('{0}\t{1}\t{2}\t{3}\t{4}\n'.format(deltav,a_error,e_error,i_error,raan_error))
        #fitness = 25*deltav + a_error + 41666.67*e_error + 100*i_error + 50*raan_error 
    # For case 2  (SS-LEO)
        a_error = np.absolute(sv_kep[0]-7080)
        e_error = np.absolute((sv_kep[1]) - 0.00112)
        i_error =  np.absolute((sv_kep[2] - 98.2))
        raan_error = np.absolute((sv_kep[3]) - 0)
        #print('{0}\t{1}\t{2}\t{3}\t{4}\n'.format(deltav,a_error,e_error,i_error,raan_error))
        fitness = 25*deltav + a_error + 7142.86*e_error + 10*i_error + 0.40*raan_error 
        #print(fitness)
    else:
        #fitness = np.absolute((LA.norm(x[0:3]) - 42164)/10) + np.absolute((LA.norm(x[3:6]) - 3.07466458018)/0.01)
        
        # For the first test
        #fitness = np.absolute((LA.norm(x[0:3]) - 42164))
        
        # For the second test
        fitness = np.absolute((LA.norm(x[0:3]) - 7080))
    #print('Fitness: {0} at {1}.\n'.format(fitness,process_number))    
     # Returning the final vector to the calling method
    end = time.time()
    #print("Process {} has ended at {}".format(process_number, end))
    sys.stdout.flush()
    queue.put((process_number, fitness))        
    
    
    
def fitness_function(X,final_vector):    
    # Call the fitness function to calculate the fitness of each solution
    #rearrange
    
    candidates = np.zeros((pop_size,6))
    for i in range(pop_size):
        for j in range(pop_size):
            if final_vector[j][0] == i:
                candidates[i] = final_vector[j][1]
    
#     print(final_vector)
#     print(candidates)
#     sys.exit()
    queue = mp.Queue()
    processes = [mp.Process(target=fitness_calculator, args=(x, queue, candidates[x], X[x][:])) \
                 for x in range(pop_size)]
    for p in processes:
        p.start()

    # Unhash the below if you run on Linux (Windows and Linux treat multiprocessing
    # differently as Windows lacks os.fork())
    for p in processes:
        p.join()

    fitness_results = [queue.get() for p in processes]
    #print('fitness_results : {}\n'.format(fitness_results))
    #return results in arranged order
    candidates_fitness = np.zeros((pop_size,1))
    for i in range(pop_size):
        for j in range(pop_size):
            if fitness_results[j][0] == i:
                candidates_fitness[i,0] = fitness_results[j][1]
#     print(fitness_results)
#     print(candidates_fitness)
#     sys.exit()            
    return(candidates_fitness)

def calc_initial_vector(x,i,sv_old):
    sv_init = np.zeros(6)
    tmp = kep2cart(sv_old,x[i*4+3]) 
    #print('sv cart is {0} before delta v for impulse: {1} . \n'.format(tmp,i))
    sv_cart = [tmp[0],tmp[1],tmp[2], tmp[3] + x[i*4], tmp[4] + x[i*4+1],tmp[5] + x[i*4 +2]]
    #print('sv_cart after deltav: {0} at impulse {1} '.format(sv_cart,i))
    sv_init = cart2kep(sv_cart[0:3], sv_cart[3:6])
    #print('sv kep is {0} after delta v for impulse: {1} . \n'.format(sv_init,i))
    #print('sv_cart is {0} after delta v for impulse: {1} . \n'.format(sv_cart,i))    
    return(sv_init)

def func_main(process_number, x, queue = None):
    
    start = time.time()
    #print("Process {} has started at {}".format(process_number, start))
    sys.stdout.flush()
    
    sol_id = process_number + 1
    p = './OrbitIntegrator/%d' % sol_id
    
    # check for path existence
    folder_already_exists_check(p)
    make_sure_path_exists(p)
    
    # To copy the integrator
    subprocess.call('cp -r ./OrbitIntegrator/Orbit_Sim/* ./OrbitIntegrator/%d' % sol_id , shell = True)
    
    # loop for multiple impulse
    for i in range(no_of_impulses):
        if i == 0:
            sv_init_keplar = initial_state_vector
            
        # after adding delta v (impulse)
        sv_instant = calc_initial_vector(x,i, sv_init_keplar)
        #print('sv_instant : {}\n'.format(sv_instant))
        
        # call to save the state vector in val.txt in the respective folder
        fo = open("./OrbitIntegrator/%d/val.txt" % sol_id, "wb")
        np.savetxt(fo , sv_instant, fmt='%5.6f', delimiter=' ', newline=' ')
        fo.close()
    
        # call to edit input.cst 
        subprocess.call('./enter_vector_intergrator.sh %d' % sol_id , shell = True)
    
        # call to run each instance of integrator
        try:
            subprocess.call('cd ./OrbitIntegrator/%d/ && ./orbitsim64_rhel5' % sol_id, shell = True)
        except OSError as exception:
            if exception.errno != errno.EEXIST:
                raise    
    
        # call to read final state vector
        subprocess.call('./read_final_sv.sh %d' % sol_id, shell = True)
        sv_final = np.genfromtxt('./OrbitIntegrator/%d/output/FinalVector.txt' % sol_id )
        
        # call to destroy val.txt for next step
        subprocess.call('./delete_tmp_file.sh %d' % sol_id , shell = True)
    
        # here go for next impulse to calclate the final orbit
        if i != no_of_impulses-1:
            sv_init = sv_final
            #print('sv cartesian after impulse 0 : {}\n'.format(sv_init))
            sv_init_keplar = cart2kep(sv_init[0:3],sv_init[3:6])
                    
            #print('sv keplar after impulse 0 : {}\n'.format(sv_init_keplar))
    
    # print the sv_final
   # if process_number == 1:
        #print('cartesian final: {}\n'.format(sv_final))
    #print(LA.norm(sv_final[0:3]))
    tmp_kep = cart2kep(sv_final[0:3],sv_final[3:6])
   # if process_number == 1:
        #print('Keplarian final: {}\n'.format(tmp_kep))
    # remove the directory created
    remove_temp_dir(p)
    
    # Returning the final vector to the calling method
    end = time.time()
    #print("Process {} has ended at {}".format(process_number, end))
    sys.stdout.flush()
    
    if queue is not None:
        queue.put((process_number, sv_final)) 
    else:
        return(sv_final)
    
def integrator(X): 
    
    queue = mp.Queue()
    processes = [mp.Process(target=func_main, args=(x, X[x], queue)) for x in range(pop_size)]
    
    for p in processes:
        p.start()

    # Unhash the below if you run on Linux (Windows and Linux treat multiprocessing
    # differently as Windows lacks os.fork())
    for p in processes:
        p.join()

    results = [queue.get() for p in processes]
    return(results)

            
def multiproc_master():
    
   # global X
    queue = mp.Queue()
    
    if no_of_impulses == 1:
        es = cma.CMAEvolutionStrategy([2,2,2,0], 0.2, {'seed':234, 'CMA_mu':cmu_mu, 'popsize':pop_size,
                                'bounds': [[-10, -10, -10, -1], [10, 10, 10, 1]],
                                                    'ftarget': -np.inf, 'verb_append':1})
    elif no_of_impulses == 2:
        es = cma.CMAEvolutionStrategy([0,0,2,0,0,0,2,180], 0.2, {'seed':234, 'CMA_mu':cmu_mu, 'popsize':pop_size,
                                'bounds': [[-4, -4, -4, 0, -4, -4, -4, 180], [4, 4, 4, 180, 4, 4, 4, 360]],
                                                    'ftarget': 0, 'maxiter':200})
        
    while not es.stop():
        X = []
        while len(X) < es.popsize:
            curr_fit = None
            while curr_fit in (None, np.NaN):
                x = es.ask(1)[0]
                #print(x)
                #if (LA.norm(x[0:3]) < 3 and LA.norm(x[4:7]) < 2):
                    #print('Dv1 = {0}, Dv2 = {1}\n'.format(LA.norm(x[0:3]),LA.norm(x[4:7])))
                curr_fit = 1 # might return np.NaN
                    #print(curr_fit)
            X.append(x)
            #fit.append(curr_fit)
        #X = es.ask()
        #print(X)
        final_vector = integrator(X)
        fit = fitness_function(X, final_vector)
    #check for acceptence
        es.tell(X, fit)
        es.disp(5)
    print('X: {0} and Fitness: {1}.\n'.format(X,fit))  
    
def multiproc_master_SS_LEO_orbit_raising():
    
   # global X
    queue = mp.Queue()

#    elif no_of_impulses == 2:
    es = cma.CMAEvolutionStrategy([0,0,0.02,0,0,0,0.02,180], 0.2, {'seed':234, 'CMA_mu':cmu_mu, 'popsize':pop_size,
                                'bounds': [[-0.5, -0.5, -0.5, 0, -0.5, -0.5, -0.5, 180], \
                                           [0.5, 0.5, 0.5, 180, 0.5, 0.5, 0.5, 360]],
                                                    'ftarget': 0, 'maxiter':100})
        
    while not es.stop():
        X = []
        while len(X) < es.popsize:
            curr_fit = None
            while curr_fit in (None, np.NaN):
                x = es.ask(1)[0]
                #print(x)
                final_vector = func_main(1,x)
                f_k = cart2kep(final_vector[0:3], final_vector[3:6])
                if (np.absolute(f_k[1] - 0.00112) < 0.0001 and np.absolute(f_k[2] - 98.2) < 0.1 ):
                    #print('Dv1 = {0}, Dv2 = {1}\n'.format(LA.norm(x[0:3]),LA.norm(x[4:7])))
                    curr_fit = 1 # might return np.NaN
                    #print(curr_fit)
                    
            X.append(x)
            curr_fit = fitness_function(X, final_vector)
            fit.append(curr_fit)
        #X = es.ask()
        #print(X)
        #final_vector = integrator(X,popsize)
        #fit = fitness_function(X, final_vector)
    #check for acceptence
        es.tell(X, fit)
        es.disp(5)
    print('X: {0} and Fitness: {1}.\n'.format(X,fit)) 
    
def testing():        
    #Testing    
    a = np.array([0, 0, 2.33680, 0, 0, 0, -1.43390, 180])
#     #print(a)
    X = np.tile(a, (pop_size, 1))
#     #print(X)
    #X = es.ask()
    final_vector = integrator(X)
    fit = fitness_function(X, final_vector)

    #print('X: {0} and Fitness: {1}.\n'.format(X,fit))
    
if __name__ == '__main__':
    # for converting the keplerian to cartesian
    #initial_state_vector = kep2cart(initial_state_vector, 0)
    #split_jobs = multiproc_master()
    split_jobs = multiproc_master_SS_LEO_orbit_raising()
    #split_jobs = testing()
    #print(split_jobs)
    print("Tarun")
